In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys, os, pathlib
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'

import numpy as np
import xarray as xr
import pandas as pd
import nibabel as nib
import torch

sys.path.append('../..')
import project

torch.cuda.is_available()

True

In [10]:
%autoreload
data_root = pathlib.Path('/ocean/projects/asc170022p/mtragoza/lung-project/data/COPDGene')
data_file = data_root / 'sample1000_2025-07-22.csv'

# load sampled data file
df_sampled = pd.read_csv(data_file, sep='\t', low_memory=False)
df_sampled

,sid,ccenter,kernel,Emphysema,pctEmph,pctEmph_Slicer,FEV1pp_utah,FVCpp_utah,FEV1_FVC_utah,finalGold,catEmph
0,16514P,TEM,STD,0,0.3373,0.3147,77.6,88.9,0.71,-1.0,normal
1,20748Q,UAB,STD,0,3.6045,3.4085,73.2,75.8,0.75,-1.0,normal
2,11007Z,USD,STD,0,0.3126,0.2867,66.7,108.0,0.47,2.0,normal
3,14771Z,HPR,STD,0,0.3160,0.2948,94.1,97.0,0.73,0.0,normal
4,13651K,UIA,STD,0,2.0807,1.9551,85.1,88.0,0.75,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...
995,20519B,DUK,STD,1,30.1942,29.6440,19.9,69.2,0.22,4.0,severe
996,12294H,UAB,STD,1,40.3908,39.5845,43.3,86.0,0.37,3.0,severe
997,23123R,TEM,STD,0,23.6838,22.7617,42.8,72.7,0.45,3.0,severe
998,16546C,UIA,STD,1,42.6116,41.8307,58.6,99.6,0.43,2.0,severe


In [11]:
dataset = project.copdgene.COPDGeneDataset(df_sampled, data_root)
dataset

In [12]:
row, visit = dataset[0]
row

sid               16514P
ccenter              TEM
kernel               STD
Emphysema              0
pctEmph           0.3373
pctEmph_Slicer    0.3147
FEV1pp_utah         77.6
FVCpp_utah          88.9
FEV1_FVC_utah       0.71
finalGold           -1.0
catEmph           normal
Name: 0, dtype: object

In [13]:
variant = 'RAW'
source_name = visit.build_image_name(state='INSP', recon='STD')
target_name = visit.build_image_name(state='EXP', recon='STD')

In [14]:
%autoreload
project.deformation.create_deformation_field(visit, variant, source_name, target_name)

(512, 512, 383)
(512, 512, 537)
Compute fixed MIND features ... finished (1.60 s).
Stage 1/2
    search radius: 16
      cube length: 6
     quantisation: 2
     patch radius: 3
        transform: n
    Compute moving MIND features ... finished (1.14 s).
    6500 fixed keypoints extracted (1.57 s).
    Compute forward marginals ... finished (3.62 s).
    Compute symmetric backward marginals ... finished (3.60 s).
    Dense thin plate spline interpolation ... finished (4.10 s).

Stage 2/2
    search radius: 8
      cube length: 3
     quantisation: 1
     patch radius: 2
        transform: n
    Compute moving MIND features ... finished (1.56 s).
    13325 fixed keypoints extracted (1.54 s).
    Compute forward marginals ... finished (3.85 s).
    Compute symmetric backward marginals ... finished (4.09 s).
    Dense thin plate spline interpolation ... finished (8.34 s).



In [20]:
%autoreload
disps = dataset[0][0].load_displacements(image_dir, disp_dir)
disps

../data/COPDGene/Images/10009Y/Phase-1/CorrField/10009Y_EXP_SHARP_NJC_COPD/10009Y_INSP_SHARP_NJC_COPD.nii.gz True
../data/COPDGene/Images/10009Y/Phase-1/CorrField/10009Y_EXP_STD_NJC_COPD/10009Y_INSP_STD_NJC_COPD.nii.gz True
../data/COPDGene/Images/10009Y/Phase-1/CorrField/10009Y_INSP_SHARP_NJC_COPD/10009Y_EXP_SHARP_NJC_COPD.nii.gz True
../data/COPDGene/Images/10009Y/Phase-1/CorrField/10009Y_INSP_STD_NJC_COPD/10009Y_EXP_STD_NJC_COPD.nii.gz True


[<xarray.DataArray '10009Y_EXP_SHARP_NJC_COPD' (mov_image_name: 1, x: 256,
                                                y: 256, z: 256, component: 3)> Size: 403MB
 array([[[[[ -5.74974394,  20.83345604, -32.03852081],
           [ -5.86159658,  20.83122253, -31.97860527],
           [ -5.97345066,  20.82899284, -31.91869354],
           ...,
           [-23.53556442, -10.31339359,   0.48266104],
           [-23.5219059 , -10.38676262,   0.66177827],
           [-23.50825119, -10.46012878,   0.8408913 ]],
 
          [[ -5.68092251,  21.0078373 , -32.01590347],
           [ -5.7915616 ,  21.00687218, -31.95598793],
           [ -5.90220213,  21.00590706, -31.8960743 ],
           ...,
           [-23.52248383, -10.19979191,   0.52757078],
           [-23.50871468, -10.27105045,   0.70763803],
           [-23.49494553, -10.34230614,   0.88770103]],
 
          [[ -5.61210108,  21.18221855, -31.99328423],
           [ -5.72152615,  21.18251991, -31.93336868],
           [ -5.83095312, 